In [2]:
cd drive/MyDrive/

/content/drive/MyDrive


In [3]:
import numpy as np
import pandas as pd
from statsmodels.regression.quantile_regression import QuantReg
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.metrics import mean_absolute_error

train = pd.read_csv('./dacon_태양광발전량예측/preprocessed_data/pre_train_0118.csv')
test = pd.read_csv('./dacon_태양광발전량예측/preprocessed_data/pre_test_0118.csv')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
include_column = ['Hour', 'DHI', 'WS', 'RH', 'T', 'time', 'time_period', 'DHI+DNI', 'DHI_7_mean', 'DHI_7_max', 'DHI_7_min', 'DHI_7_var', 'DHI_7_max-min', 'DHI_7_Q1', 'DHI_7_Q3', 'DNI_7_mean', 'DNI_7_max', 'DNI_7_min', 'DNI_7_var', 'DNI_7_max-min', 'DNI_7_Q1', 'DNI_7_Q3', 'WS_7_mean', 'WS_7_max', 'WS_7_min', 'WS_7_var', 'WS_7_max-min', 'WS_7_Q1', 'WS_7_Q3', 'RH_7_mean', 'RH_7_max', 'RH_7_min', 'RH_7_var', 'RH_7_max-min', 'RH_7_Q1', 'RH_7_Q3', 'T_7_mean', 'T_7_max', 'T_7_min', 'T_7_var', 'T_7_max-min', 'T_7_Q1', 'T_7_Q3', 'TARGET_7_mean', 'TARGET_7_max', 'TARGET_7_min', 'TARGET_7_var', 'TARGET_7_max-min', 'TARGET_7_Q1', 'TARGET_7_Q3', 'day_sum_TARGET', 'day_mean_target', 'hour_day_TARGET_ratio', 'DHI_slot_ratio', 'DNI_slot_ratio', 'WS_slot_ratio', 'RH_slot_ratio', 'TARGET_slot_ratio', 'DHI_bfr_time_var', 'DNI_bfr_time_var', 'WS_bfr_time_var', 'RH_bfr_time_var', 'T_bfr_time_var', 'TARGET_bfr_time_var', 'DHI_bfr_time_ratio', 'DNI_bfr_time_ratio', 'WS_bfr_time_ratio', 'RH_bfr_time_ratio', 'TARGET_bfr_time_ratio', 'day_length_ratio', 'DHI_bfr_1_diff', 'DNI_bfr_1_ratio', 'DNI_bfr_1_diff', 'WS_bfr_1_ratio', 'RH_bfr_1_ratio', 'T_bfr_1_ratio', 'TARGET_bfr_1_ratio', 'TARGET_bfr_1_diff', 'DHI_bfr_2_diff', 'DNI_bfr_2_ratio', 'DNI_bfr_2_diff', 'WS_bfr_2_ratio', 'RH_bfr_2_ratio', 'TARGET_bfr_2_ratio', 'TARGET_bfr_2_diff']

train = train[include_column+['Target1','Target2']]
test = test[include_column]

In [ ]:
def quantile_loss(q, y, f):
    e = y - f
    return np.maximum(q * e, (q - 1) * e)

## hyper parameter list
max_iter = [10000,15000]

def lin(q, train_x, train_y, valid_x, valid_y) :
  model = QuantReg(train_y, train_x).fit(q=q, kernel='gau', max_iter=m)
  val = pd.Series(model.predict(valid_x).round(2))
  return model, val

def fit_lin_val(train_x, train_y, valid_x, valid_y) :
  models = []
  val_scores = 0
  for q in quantiles :
    print(q)
    model, val = lin(q, train_x, train_y, valid_x, valid_y)
    models.append(model)
    val_scores += quantile_loss(q, valid_y, val).mean()
  return models, val_scores


for m in max_iter :
    print(m)
    
    f_scores = 0
    spliter = ShuffleSplit(n_splits=5, test_size=0.3, random_state=8)
    
    for train_index, valid_index in spliter.split(list(train.index)) :
      X_train_1,Y_train_1,X_valid_1,Y_valid_1 = train.iloc[train_index,:-2],train.iloc[train_index,-2], train.iloc[valid_index,:-2], train.iloc[valid_index,-2]
      X_train_2,Y_train_2,X_valid_2,Y_valid_2 = train.iloc[train_index,:-2],train.iloc[train_index,-1], train.iloc[valid_index,:-2], train.iloc[valid_index,-1]
      
      quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
      
      # Target1
      models_1, val_scores_1 = fit_lin_val(X_train_1, Y_train_1, X_valid_1, Y_valid_1)
      # Target2
      models_2, val_scores_2 = fit_lin_val(X_train_2, Y_train_2, X_valid_2, Y_valid_2)
      
      f_scores += (val_scores_1 + val_scores_2)/18
      print((val_scores_1 + val_scores_2)/18)     
      
    print('f_scores :',f_scores)